# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [ ]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [4]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010 driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [5]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [6]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [7]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton from the Mercedes team.


### 🔹 Observations

The 2nd time it had right info but wrong format.

🧠 Why did this happen?

Because:

- The model is probabilistic, not rule-based — it tries to follow patterns but may “drift” if it’s confident it knows a better way to express the answer.

- Your prompt didn’t explicitly tell it “**always answer** in the same format.” It only showed examples.

- The **`temperature=1`** parameter adds randomness → more creativity, less consistency.

By Sofia

---

We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [8]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [10]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [19]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [28]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You like Berlin and you are a historian.

     What are the top 3 clubs?
     Club: RSO.
     Area: Schoneweide.

     Club: Humbolthain
     Area: Gesundbrunnen.

     Club: Viktoria park
     Area: Kreuzberg.

     What are the top 3 restaurants?
     restaurant: Umami.
     Area: Mehringdam

     restaurant: NC Kebap.
     Area: Baumschulenweg
     """}
]
print(return_OAIResponse("What are the top 3 famous parks", context_user))

1. Tiergarten: Located in the Mitte district, Tiergarten is one of Berlin's most famous and beloved parks. It is a large green space with walking paths, gardens, and the iconic Victory Column.

2. Tempelhofer Feld: Tempelhofer Feld is a former airport turned public park located in the Tempelhof-Schöneberg district. It offers vast open spaces for activities such as cycling, jogging, kite flying, and picnicking.

3. Volkspark Friedrichshain: Situated in the Friedrichshain district, this park is the oldest public park in Berlin and features a charming rose garden, a fairytale fountain, and a hill with great views of the city.


In [31]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You like Berlin and you are a historian.\n\n'},
    {'role':'user', 'content':'What are the top 3 clubs?'},
    {'role':'assistant', 'content':"""Club: RSO. \nArea: Schoneweide. \nClub: Humbolthain \nArea: Gesundbrunnen. \nClub: Viktoria park \nArea: Kreuzberg."""},
    {'role':'user', 'content':'What are the top 3 restaurants?'},
    {'role':'assistant', 'content':"""restaurant: Umami \nArea: Mehringdam \nrestaurant: NC Kebap. \n.
      """},
]

print(return_OAIResponse("What are the top 3 famous parks?", context_user))

Park: Tiergarten 
Location: Mitte
Park: Tempelhofer Park
Location: Tempelhof-Schöneberg
Park: Volkspark Friedrichshain 
Location: Friedrichshain-Kreuzberg


In [32]:
context_user = [
    {'role':'system', 'content':'You like Berlin and you are a historian.\n\n'},

    {'role':'user', 'content':'What are the top 3 clubs?'},
    {'role':'assistant', 'content':"""Club: RSO. \nArea: Schoneweide. \nClub: Humbolthain \nArea: Gesundbrunnen. \nClub: Viktoria park \nArea: Kreuzberg."""},

    {'role':'user', 'content':'What are the top 3 restaurants?'},
    {'role':'assistant', 'content':"""Restaurant: Umami \nArea: Mehringdam \nRestaurant: NC Kebap \nArea: Baumschulenweg."""},

    {'role':'user', 'content':'What are the top 3 museums?'},
    {'role':'assistant', 'content':"""Museum: Pergamonmuseum \nArea: Mitte \nMuseum: Neues Museum \nArea: Mitte \nMuseum: Deutsches Historisches Museum \nArea: Mitte."""},
]

print(return_OAIResponse("What are the top 3 famous theaters?", context_user))


Theater: Deutsches Theater 
Area: Mitte 
Theater: Berliner Ensemble 
Area: Mitte 
Theater: Volksbuhne 
Area: Mitte


# Report: Few-Shot Prompting for Berlin Locations

## Objective
The goal of this exercise was to explore how few-shot prompting affects GPT’s ability to provide structured information about Berlin, including clubs, restaurants, parks, and theaters. I wanted to see how example formatting influences consistency and accuracy.

---

## Method
1. **Few-shot / instruction-only without following GPT model:**  
   Initially, I gave GPT a system message describing that it should act as a historian familiar with Berlin. I asked questions about top clubs, restaurants, or parks without providing examples.  
   - **Result:** The model seemed to give correct answers, but the formatting varied, sometimes using "is" instead of a colon (`:`) to separate fields.

2. **Few-shot prompting following the GPT model:**  
   I then provided one or two examples for clubs and restaurants in the following format:  
   Item: Name
   Area: Location

- **Observation:** Model responses started matching the provided structure.

3. **Adding a third example (museums):**  
I extended the context to include museums as a third example, maintaining the same `Item` / `Area` pattern. Then I asked for top theaters.

---

## Findings / Observations
- **Formatting consistency improves with examples:**  
- Few-shot answers varied in punctuation and layout.  
- Few-shot answers following the GPT model (system, assistant, user)followed the `Item: ... \nArea: ...` pattern closely.
- **Model sometimes repeats areas:**  
- Example: All theaters returned “Area: Mitte” even if the actual location differs slightly — minor hallucination likely due to limited examples.
- **Correct entities generally returned:**  
- The model correctly listed known parks, and theaters, showing strong recall when examples were given.  
- **More examples reduce errors:**  
- Adding a third example (museums) helped the model generalize formatting to new categories, though content accuracy still depends on its knowledge cut-off.

---

## Key Learnings
1. **Few-shot examples are powerful:** Providing structured examples guides GPT to produce consistent and predictable output.  
2. **Explicit formatting matters:** Repeating the desired output format in multiple examples reduces “creative” deviations.  
3. **Hallucination is still possible:** GPT may repeat areas or invent minor details; careful review is needed for critical tasks.  
4. **Roles enhance learning:** Using `system`, `user`, and `assistant` roles aligns with the model’s training and improves pattern adoption.  
5. **Zero-shot vs few-shot:** Zero-shot can produce correct answers, but few-shot ensures structural consistency and reliability in formatting.

---

## Conclusion
Few-shot prompting is an effective method for eliciting structured and consistent responses from GPT. By providing multiple examples, the model learns the desired format in real-time during inference. However, while formatting consistency improves, minor factual hallucinations may still occur, so outputs should be verified for accuracy in real-world applications.

